# Setup

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights
import torchvision.transforms as T
from PIL import Image
from sklearn.linear_model import Ridge

from google.colab import drive
drive.mount('/content/drive/')

# Data

In [ ]:
RootDir = '/content/drive/MyDrive/STAT288_Project/'

In [ ]:
def augment_data_rotation(x, y):
    augmented_x = []
    augmented_y = []
    for i in range(len(x)):
        curr = x[i]
        for _ in range(4):
            # flips
            flip_x = np.flip(curr, axis=0)
            flip_y = np.flip(curr, axis=1)
            flip_xy = np.flip(flip_x, axis=1)
            augmented_x.append(flip_x)
            augmented_y.append(y[i])
            augmented_x.append(flip_y)
            augmented_y.append(y[i])
            augmented_x.append(flip_xy)
            augmented_y.append(y[i])

            # rotations
            augmented_x.append(curr)
            augmented_y.append(y[i])
            curr = np.rot90(curr)
    return (augmented_x, augmented_y)

In [ ]:
def augment_rotate(x):
    augmented_x = []
    curr = x
    for _ in range(4):
        # rotations
        augmented_x.append(curr)
        curr = torch.rot90(curr, dims=[1, 2])
    return augmented_x

In [ ]:
# Load data
x_train = np.load(RootDir + 'SpaceNet/sat_train.npy').astype('float32')
init_y_train = np.load(RootDir + 'SpaceNet/bul_train.npy').astype('float32')
x_test = np.load(RootDir + 'SpaceNet/sat_test.npy').astype('float32')
init_y_test = np.load(RootDir + 'SpaceNet/bul_test.npy').astype('float32')

print("x_train shape", x_train.shape)
print("init_y_train shape", init_y_train.shape)
print("x_test shape", x_test.shape)
print("init_y_test shape", init_y_test.shape)

y_train = np.sum(init_y_train, axis=(1, 2, 3))
y_test = np.sum(init_y_test, axis=(1, 2, 3))
print("y_train shape", y_train.shape)
print("y_test shape", y_test.shape)

In [ ]:
# Rotate the original images
augmented_x_train, augmented_y_train = augment_data_rotation(x_train, y_train)
augmented_x_test, augmented_y_test = augment_data_rotation(x_test, y_test)

augmented_x_train = np.asarray(augmented_x_train)
augmented_y_train = np.asarray(augmented_y_train)
augmented_x_test = np.asarray(augmented_x_test)
augmented_y_test = np.asarray(augmented_y_test)

print("augmented_x_train shape", augmented_x_train.shape)
print("augmented_y_train shape", augmented_y_train.shape)

In [ ]:
transform = T.ToPILImage()
x_train_imgs = []
for i in range(len(x_train)):
    x_train_imgs.append(transform(x_train[i]))

x_test_imgs = []
for i in range(len(x_test)):
    x_test_imgs.append(transform(x_test[i]))

# Model

In [ ]:
# Get weights
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.eval()

In [ ]:
# Preprocess data
preprocess = weights.transforms()
transformed_x_train = []
for i in range(len(x_train_imgs)):
    transformed_x_train.append(preprocess(x_train_imgs[i]).unsqueeze(0))

transformed_x_test = []
for i in range(len(x_test_imgs)):
    transformed_x_test.append(preprocess(x_test_imgs[i]).unsqueeze(0))

In [ ]:
# Run inference to get latent variables
x_train_vars = []
for i in range(len(transformed_x_train)):
    x_train_vars.append(model(transformed_x_train[i]).squeeze(0))

x_test_vars = []
for i in range(len(transformed_x_test)):
    x_test_vars.append(model(transformed_x_test[i]).squeeze(0))

# Ridge

In [ ]:
# Run ridge regression
x_trains_vars_np = []
for i in range(len(x_train_vars)):
    x_trains_vars_np.append(x_train_vars[i].detach().numpy())
clf = Ridge()
clf.fit(x_trains_vars_np, y_train)

# Inference

In [ ]:
stddev = []
for i in range(len(x_test_imgs)):
    out = []
    for elt in augment_rotate(preprocess(x_test_imgs[i])):
        out.append(clf.predict([model(elt.unsqueeze(0)).squeeze(0).detach().numpy()])[0])
    stddev.append(np.std(out))
print(np.average(stddev))

In [ ]:
values = [5130.882, 4400.313, 5226.372, 3682.026]
# Plot train data
plt.figure(figsize=(24, 16))
plt.title("Training Data")
for i in range(4):
    plt.subplot(1, 4, i + 1)
    plt.title(f"Predicted Index: {values[i]}")
    plt.imshow(augmented_x_train[4 * i, :, :, :].astype('uint8'))

plt.tight_layout()
plt.show()